In [39]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import matplotlib.pylab as plt

In [3]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [4]:
# Import des questions

In [117]:
answers_grand_debat_LTE_raw = pd.read_csv('raw/large_datasets_ELF/LA_TRANSITION_ECOLOGIQUE.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_FDP_raw = pd.read_csv('raw/large_datasets_ELF/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_ORG_raw = pd.read_csv('raw/large_datasets_ELF/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_DEC_raw = pd.read_csv('raw/large_datasets_ELF/DEMOCRATIE_ET_CITOYENNETE.csv', encoding="utf-8").drop_duplicates(keep=False)

In [173]:
def keep_popular_hashtags_by_question(dataframe):
    allowed_categories_gda = []
    print(dataframe['id_gda'].unique())
    for question in dataframe['id_gda'].unique():
        allowed_categories_tuples = Counter(dataframe[dataframe['id_gda'] == question]['tag_elf']).most_common(20)
        allowed_categories_gda += [(question, element[0], element[1]) for element in allowed_categories_tuples]
    for question in dataframe['id_elf'].unique():    
        allowed_categories_tuples = Counter(dataframe[dataframe['id_elf'] == question]['tag_elf']).most_common(20)
        allowed_categories = [element[0] for element in allowed_categories_tuples]
        dataframe = dataframe[(dataframe['id_elf'] != 'question') | (dataframe['tag_elf'].isin(allowed_categories))]
    return dataframe, allowed_categories_gda

def process_LGD_dataset(raw_dataframe):
    """
    """
    answers_annotees = pd.read_csv('raw/actions.csv', encoding="utf-8").rename(columns={'Contribution': 'reference'})
    print(raw_dataframe.shape)
    raw_dataframe = pd.merge(answers_annotees, raw_dataframe, on='reference')
    if raw_dataframe.shape[0] == 0:
        print("Empty records when matching on annotations")
        return None, None
    
    questions_dictionary = pd.read_csv('raw/match_questions_ELF_LGA.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, questions_dictionary, left_on='Question', right_on='id_gda')
    print(raw_dataframe.shape)

    gda_dictionary = pd.read_csv('raw/matching_gda_id1_id2.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, gda_dictionary, left_on='id_gda', right_on='id1_gda')
    print(raw_dataframe.shape)
    
    raw_dataframe["corpus"] = None
    
    for index, row in raw_dataframe.iterrows():
        raw_dataframe.at[index,'corpus'] = raw_dataframe.loc[index][raw_dataframe.filter(like=row['id2_gda']).columns[0]]

    raw_dataframe = raw_dataframe[['reference', 'id_elf', 'corpus', 'Categorie', 'Annotateur', 'id_gda']]
    
    tags_dictionary = pd.read_csv('raw/matching_tags_lga_elf.csv', sep=';')
    raw_dataframe = pd.merge(raw_dataframe, tags_dictionary, left_on=['id_gda', 'Categorie'], right_on=['id_lga', 'tag_lga'])
    print(raw_dataframe.shape)
    
    raw_dataframe, allowed_categories_gda = keep_popular_hashtags_by_question(raw_dataframe)
    print(raw_dataframe.shape)
    
    raw_dataframe = pd.DataFrame({'count' : raw_dataframe.groupby( ['reference', 'tag_elf', 'corpus', 'id_elf'] ).apply(lambda x: x.Annotateur.nunique())}).reset_index()
    print(raw_dataframe.shape)
    
    return raw_dataframe.rename(columns={'reference': 'contribution', 'corpus': 'answer', 'tag_elf': 'target', 'id_elf' : 'question'}), allowed_categories_gda
    

In [174]:
answers_grand_debat_GDA_DEC, allowed_categories_gda_DEC = process_LGD_dataset(answers_grand_debat_DEC_raw)
answers_grand_debat_GDA_FDP, allowed_categories_gda_FDP = process_LGD_dataset(answers_grand_debat_FDP_raw)
# answers_grand_debat_GDA_ORG, allowed_categories_gda_ORG = process_LGD_dataset(answers_grand_debat_ORG_raw)
answers_grand_debat_GDA_LTE, allowed_categories_gda_LTE = process_LGD_dataset(answers_grand_debat_LTE_raw)

(73746, 48)
(5915, 55)
(5915, 57)
(2878, 9)
[127]
(2878, 9)
(926, 5)
(126165, 19)
(23033, 26)
(23033, 28)
(17454, 9)
[162 163]
(17454, 9)
(6804, 5)
(99343, 27)
(20296, 34)
(20296, 36)
(17896, 9)
[149]
(17896, 9)
(5334, 5)


In [175]:
answers_grand_debat_GDA = pd.concat([answers_grand_debat_GDA_FDP, answers_grand_debat_GDA_DEC, answers_grand_debat_GDA_LTE])
answers_grand_debat_GDA['source'] = 'GDA'

In [185]:
answers_grand_debat_ELF = pd.read_csv("raw/elf_tags_sample_v2.csv", sep=";").rename(columns={'reference': 'contribution', 'corpus': 'answer', 'tag': 'target', 'id_elf' : 'question'})
answers_grand_debat_ELF['source'] = 'ELF'

In [187]:
answers_grand_debat_ELF.head()

,contribution,target,answer,question,count,source
0,42 431,Diminution salaires et suppression des avantag...,Les dépenses du gouvernement et les dépenses a...,FI9,1,ELF
1,13 610,Développement voiture électrique,Il faut arrêter certaines taxes et accompagner...,TE13,1,ELF
2,12 847,"Lutte évasion, fraude, optimisation",Mieux lutter contre la fraude interdire les pa...,FI8,1,ELF
3,13 630,Incompétence et inefficacité,Trop de personnes inutiles alors que certains ...,OE9,1,ELF
4,13 860,Lenteur administrative,Très long ! &&Demander moins de paperasse pour...,OE9,1,ELF


In [188]:
answers_grand_debat = pd.concat([answers_grand_debat_ELF, answers_grand_debat_GDA])
Counter(answers_grand_debat['target']).most_common(20)

[('Tri sélectif, compost, réduction déchets', 1787),
 ('Transports plus verts ou réduits', 1462),
 ('Autre / Passer', 1373),
 ('Consommation réduite ou vertueuse (bio, circuit court, local)', 1208),
 ('Plus de transparence', 950),
 ('Réduction consommation énergie, eau', 939),
 ('Impôt symbolique pour tous', 868),
 ('Plus de tranches IR progressives', 750),
 ('Régime alimentaire', 742),
 ('Lutte évasion, fraude, optimisation', 540),
 ('racisme, xénophobie, discriminations religieuses', 513),
 ("Plus d'information sur la dépense des impôts", 446),
 ('Revoir niches', 445),
 ('sexisme', 444),
 ('Rétablir ISF', 443),
 ('Plus de pédagogie', 442),
 ('Meilleure isolation, changement chauffage/chaudière', 409),
 ('Application loi 1905 / Séparation stricte Etat-religions', 408),
 ('Taxer les riches', 352),
 ('Cultiver/Jardiner bio, sans pesticide', 323)]

In [189]:
answers_grand_debat_sample = answers_grand_debat.groupby( ['question', 'answer'] ).apply(lambda x: x.target.unique()).reset_index()

In [190]:
answers_grand_debat_sample = answers_grand_debat_sample.rename(columns={0: 'target'})

In [191]:
answers_grand_debat_sample.head()

,question,answer,target
0,DC11,* il faut faire plus de sensibilisation (surto...,"[éducation citoyenne de la jeunesse, Autre / P..."
1,DC11,- Interdire les sondages en période d'élection...,"[nouveau mode de scrutin (proportionnelle, etc.)]"
2,DC11,- Remettre les législatives AVANT la président...,[Autre / Passer]
3,DC11,- interdire toute possibilité de réélection ap...,"[prise en compte du vote blanc, éducation cito..."
4,DC11,- modernisation des outils de communication et...,"[prise en compte du vote blanc, vote via nouve..."


## Lemmatisation

In [203]:
# spaCy based imports
import spacy
nlp = spacy.load('fr_core_news_sm')

In [196]:
# Usual imports 
from tqdm import tqdm, tqdm_notebook

# NLP imports
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import stop_words
import re, sys
from gensim.models import phrases, Phrases

In [199]:
sw = stop_words.get_stop_words(language='fr')

In [200]:
answers_grand_debat_sample['tokens'] = answers_grand_debat_sample.answer.map(lambda text: [w for w in text.replace('.', '').replace(',', '').replace('!', '').split() if w])
answers_grand_debat_sample['tokens'] = answers_grand_debat_sample.answer.map(lambda text: word_tokenize(text))
answers_grand_debat_sample['tokens_clean'] =  answers_grand_debat_sample.tokens.map(
                                    lambda tok: [t.lower() for t in re.split(" ", re.sub(r"(\W+|_|\d+)", " ", " ".join(tok))) \
                                                 if t.lower() not in sw and len(t)>1]
                        )
answers_grand_debat_sample['tokens_clean_joined'] =  answers_grand_debat_sample.tokens_clean.map(lambda tok: ' '.join(tok))

In [ ]:
lemmatizer = FrenchLefffLemmatizer()

def lemmatize_answer(answer):
    lemmatized_answer = ''
    nlp_answer = nlp(answer)
    for token in nlp_answer:
        lemmatized_answer += lemmatizer.lemmatize(str(token)) + ' '
    return lemmatized_answer

tqdm.pandas()
answers_grand_debat_final['lemmatized_answer'] = answers_grand_debat_sample['tokens_clean_joined'].progress_apply(lemmatize_answer)

In [220]:
answers_grand_debat_final = answers_grand_debat_sample[['question', 'answer', 'lemmatized_answer', 'target']]

In [219]:
answers_grand_debat_final.head()

,question,answer,lemmatized_answer,target
0,DC11,* il faut faire plus de sensibilisation (surto...,faut plus sensibilisation surtout population j...,"[éducation citoyenne de la jeunesse, Autre / P..."
1,DC11,- Interdire les sondages en période d'élection...,interdire sondage période élection certain cit...,"[nouveau mode de scrutin (proportionnelle, etc.)]"
2,DC11,- Remettre les législatives AVANT la président...,remettre législative présidentielle inversion ...,[Autre / Passer]
3,DC11,- interdire toute possibilité de réélection ap...,interdire toute possibilité réélection après m...,"[prise en compte du vote blanc, éducation cito..."
4,DC11,- modernisation des outils de communication et...,modernisation outil communication vote prise c...,"[prise en compte du vote blanc, vote via nouve..."


In [217]:
answers_grand_debat_final.to_json('preprocessed/reponses_annotees_multilabel.json', orient = "records")